# Load modules

In [ ]:
# Setup for PyZentropy development
import sys
sys.path.append("../../")  # path to pyzentropy root

# Enable autoreload of modules
%reload_ext autoreload
%autoreload 2

# Third-party imports
import numpy as np
import pickle

# PyZentropy imports
from pyzentropy.configuration import Configuration
from pyzentropy.system import System

# Load the Fe₃Pt Data

This section loads the lowest three energy configurations for the Fe₃Pt 12-atom supercell from a pickle file. The data includes Helmholtz energies and their derivatives, entropies, and heat capacities. All input data was generated using the DFTTK package.

In [ ]:
# Load the Fe3Pt input data
with open("Fe3Pt_input.pkl", "rb") as f:
    input = pickle.load(f)

# PyZentropy Configuration Objects

This section constructs PyZentropy `Configuration` objects for each Fe₃Pt configuration. Properties at 0 K are excluded to avoid division by zero errors. For each configuration, internal energies and partition functions are calculated.

In [ ]:
# Extract basic properties from the reference configuration
number_of_atoms = input["config_0"]['number_of_atoms']
volumes = input["config_0"]['volumes']
temperatures = input["config_0"]['temperatures'] 

In [ ]:
# Initialize dictionary to hold Configuration objects
config_objects = {}

for config_name in input:
    config = input[config_name]

    # Create a Configuration object for each config and store in the dictionary
    config_objects[config_name] = Configuration(
        name=config_name,
        multiplicity=config['multiplicity'],
        number_of_atoms=number_of_atoms,
        volumes=volumes,
        temperatures=temperatures,
        # Extract Helmholtz energies and their derivatives from DFTTK results
        helmholtz_energies=config['helmholtz_energy'],
        helmholtz_energies_dV=config['helmholtz_energy_dV'],
        helmholtz_energies_d2V2=config['helmholtz_energy_d2V2'],
        # Extract entropy and heat capacity data
        entropies=config['entropy'],
        heat_capacities=config['heat_capacity']
    )

In [ ]:
# Calculate internal energies for each configuration
for config in config_objects.values():
    config.calculate_internal_energies()

# Use the ground state configuration as reference for partition function calculations
reference_helmholtz_energy = config_objects["config_0"].helmholtz_energies

# Calculate partition functions for each configuration using the reference energy
for config in config_objects.values():
    config.calculate_partition_functions(reference_helmholtz_energy)

Plot Helmholtz energy as a function of volume for a single configuration.

You can change the configuration name (e.g., "config_0") to plot other configurations.

**Available plot types:**
- "helmholtz_energy_vs_volume"
- "helmholtz_energy_vs_temperature"
- "internal_energy_vs_volume"
- "internal_energy_vs_temperature"
- "entropy_vs_volume"
- "entropy_vs_temperature"
- "heat_capacity_vs_volume"
- "heat_capacity_vs_temperature"


In [ ]:
config_objects["config_0"].plot("helmholtz_energy_vs_volume")

In [ ]:
# You can also plot for selected temperatures by passing an array of temperatures.
# The code will automatically find and use the closest available temperatures.
selected_temperatures = np.array([100, 300, 600, 900])
config_objects["config_0"].plot("helmholtz_energy_vs_volume", selected_temperatures=selected_temperatures)

In [ ]:
# You can also plot for selected volumes by passing an array of volumes.
# The code will automatically find and use the closest available volumes.
selected_volumes = np.array([100, 120, 140, 160])
config_objects["config_0"].plot("helmholtz_energy_vs_temperature", selected_volumes=selected_volumes)

# PyZentropy System Objects

This section constructs a PyZentropy `System` object using all Fe₃Pt configurations. Using the zentropy equations, the system partition function, Helmholtz energy and its derivatives, entropy, bulk modulus, heat capacities, and phase diagrams are calculated. Configuration probabilities are also determined.

In [ ]:
# Create a System object from the configuration objects
system = System(config_objects)

# Perform thermodynamic calculations for the system
system.calculate_partition_functions()
system.calculate_probabilities()
system.calculate_helmholtz_energies(reference_helmholtz_energy)
system.calculate_entropies()
system.calculate_helmholtz_energies_dV()
system.calculate_bulk_moduli()
system.calculate_helmholtz_energies_d2V2()
system.calculate_heat_capacities()

In [ ]:
# Calculate phase diagrams with specified ground state configuration. 
# This also calculates all the properties at constant pressure.
system.calculate_phase_diagrams(ground_state="config_0")

The following are example plots vs. volume or temperature. For all of these plots, you can plot for `selected_temperatures` and `selected_volumes`. The available plots are:

**Helmholtz Energy**
- "helmholtz_energy_vs_volume"
- "helmholtz_energy_vs_temperature"
- "helmholtz_energy_dV_vs_volume"
- "helmholtz_energy_dV_vs_temperature"
- "helmholtz_energy_d2V2_vs_volume"
- "helmholtz_energy_d2V2_vs_temperature"

**Entropy**
- "entropy_vs_volume"
- "entropy_vs_temperature"
- "configurational_entropy_vs_volume"
- "configurational_entropy_vs_temperature"

**Heat Capacity**
- "heat_capacity_vs_volume"
- "heat_capacity_vs_temperature"

**Bulk Modulus**
- "bulk_modulus_vs_volume"
- "bulk_modulus_vs_temperature"

**Phase Diagram**
- "vt_phase_diagram"


In [ ]:
system.plot_vt("helmholtz_energy_vs_volume")

In [ ]:
system.plot_vt("entropy_vs_temperature")

In [ ]:
system.plot_vt("heat_capacity_vs_temperature")

In [ ]:
system.plot_vt("bulk_modulus_vs_temperature")

In [ ]:
system.plot_vt("vt_phase_diagram")

The following are example plots for properties at constant pressure. The available plots are:

- "probability_vs_temperature"
- "helmholtz_energy_pv_vs_volume"
- "volume_vs_temperature"
- "CTE_vs_temperature"
- "LCTE_vs_temperature"
- "entropy_vs_temperature"
- "configurational_entropy_vs_temperature"
- "heat_capacity_vs_temperature"
- "gibbs_energy_vs_temperature"
- "bulk_modulus_vs_temperature"
- "pt_phase_diagram"


In [ ]:
system.plot_pt("pt_phase_diagram")

In [ ]:
system.plot_pt("probability_vs_temperature", ground_state="config_0", P=0)

In [ ]:
system.plot_pt("helmholtz_energy_pv_vs_volume", P=0)

In [ ]:
# You can also plot for selected temperatures by passing an array of temperatures.
# The code will automatically find and use the closest available temperatures.
selected_temperatures = np.array([100, 300, 600, 900])
system.plot_pt("helmholtz_energy_pv_vs_volume", P=0, selected_temperatures=selected_temperatures)

In [ ]:
system.plot_pt("volume_vs_temperature", P=0)

In [ ]:
system.plot_pt("LCTE_vs_temperature", P=0)

In [ ]:
system.plot_pt("entropy_vs_temperature", P=0)

In [ ]:
system.plot_pt("heat_capacity_vs_temperature", P=0)

In [ ]:
system.plot_pt("gibbs_energy_vs_temperature", P=0)